In [1]:
import pandas as pd

# GitHub raw URL 목록
urls = [
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/1.R1%7E2.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/2.RTM.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/3.R3%7E5.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/4.PLAY-IN.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/5.PLAYOFF.csv"
]

dfs = []

for url in urls:
    # 파일 이름만 추출 (예: Round1_Bahrain.csv → Round1_Bahrain)
    filename = url.split("/")[-1].replace(".csv", "")

    # CSV 읽기
    df = pd.read_csv(url)

    dfs.append(df)

# 모든 데이터 합치기
df_all = pd.concat(dfs, ignore_index=True)

In [3]:
# 타임라인 데이터 불러오기
import os
tl_df = pd.read_excel("../LCK_EventsData/timeline_events_combined.xlsx", engine='openpyxl', sheet_name="timeline_events1",
                      dtype={'date': str})
tl_df = tl_df[['gameid','date','time','player','action','target','igt_fixed','solokill', 'upset', 'upset_gameend',
               'drake_normal','voidgrub','herald','nashor','drake_elder','kill','takedown','nexus']]

In [4]:
tl_df['date'] = pd.to_datetime(tl_df['date'], format='%y%m%d')

In [5]:
# 마이크로초 단위 timestamp를 datetime 객체로 변환
# unit='us'를 사용하여 마이크로초 단위임을 지정합니다.
df_all['datetime_utc'] = pd.to_datetime(df_all['timestamp'], unit='us', errors='coerce')

# 초 이하 단위 제거
df_all['datetime_utc'] = df_all['datetime_utc'].dt.floor('s')

# UTC 시간을 KST (UTC+9)로 변환
# 먼저 UTC 타임존 정보를 추가하고, 그 다음 KST로 변환합니다.
df_all['datetime_kst'] = df_all['datetime_utc'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')

# 초 이하 단위 제거
df_all['datetime_kst'] = df_all['datetime_kst'].dt.floor('s')

#datetime_utc 열 제거
df_all = df_all.drop(columns=['datetime_utc', 'amount'])

df_all['date'] = pd.to_datetime(df_all['date'], format='%y%m%d')
df_all.head()

,time_text,timestamp,author,message,format,date,datetime_kst
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,R1~2,2025-06-04,2025-06-04 14:20:50+09:00
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,R1~2,2025-06-04,2025-06-04 14:32:59+09:00
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,R1~2,2025-06-04,2025-06-04 14:42:30+09:00
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,R1~2,2025-06-04,2025-06-04 14:49:39+09:00
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,R1~2,2025-06-04,2025-06-04 14:49:57+09:00


In [6]:
# df_all에 이벤트 열만 추가
event_types = ['drake_normal', 'voidgrub', 'herald', 'nashor', 'drake_elder', 'kill', 'takedown', 'nexus','upset', 'upset_gameend','solokill']
df_all[event_types] = 0

In [7]:
#df_all의 time_text 열 이름을 time으로 변경
df_all = df_all.rename(columns={'time_text': 'time'})

In [8]:
tl_df = tl_df[['gameid', 'date', 'player','action','target','igt_fixed', 'drake_normal', 'voidgrub', 'herald', 'nashor', 'drake_elder', 'kill', 'takedown', 'nexus','upset', 'upset_gameend','solokill']]
tl_df = tl_df.rename(columns={'igt_fixed':'time'})

In [33]:
df_all.to_csv("comments.csv", index=False)
tl_df.to_csv("tl_df.csv", index=False)

In [9]:
df_all.head()

,time,timestamp,author,message,format,date,datetime_kst,drake_normal,voidgrub,herald,nashor,drake_elder,kill,takedown,nexus,upset,upset_gameend,solokill
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,R1~2,2025-06-04,2025-06-04 14:20:50+09:00,0,0,0,0,0,0,0,0,0,0,0
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,R1~2,2025-06-04,2025-06-04 14:32:59+09:00,0,0,0,0,0,0,0,0,0,0,0
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,R1~2,2025-06-04,2025-06-04 14:42:30+09:00,0,0,0,0,0,0,0,0,0,0,0
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,R1~2,2025-06-04,2025-06-04 14:49:39+09:00,0,0,0,0,0,0,0,0,0,0,0
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,R1~2,2025-06-04,2025-06-04 14:49:57+09:00,0,0,0,0,0,0,0,0,0,0,0


In [10]:
tl_df.head()

,gameid,date,player,action,target,time,drake_normal,voidgrub,herald,nashor,drake_elder,kill,takedown,nexus,upset,upset_gameend,solokill
0,65431,2025-04-02,Pyosik,voidgrubs,NaN,0 days 03:22:53,0,1,0,0,0,0,0,0,0,0,0
1,65431,2025-04-02,Pyosik,voidgrubs,NaN,0 days 03:23:02,0,1,0,0,0,0,0,0,0,0,0
2,65431,2025-04-02,Pyosik,voidgrubs,NaN,0 days 03:23:09,0,1,0,0,0,0,0,0,0,0,0
3,65431,2025-04-02,Rich,kill,DuDu,0 days 03:25:10,0,0,0,0,0,1,0,0,0,0,0
4,65431,2025-04-02,NaN,PLATE,MID,0 days 03:25:44,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# tl_df에 kill_streak 열 추가
tl_df['kill_streak'] = 0

# 날짜별로 그룹화하여 처리
for date in tl_df['date'].unique():
    date_mask = tl_df['date'] == date
    date_df = tl_df[date_mask].copy()
    
    # kill 이벤트만 필터링
    kill_events = date_df[date_df['kill'] == 1].sort_values('time')
    
    if len(kill_events) < 3:
        continue
    
    # 각 킬에 대해 10초 이내의 킬 개수 확인
    for idx in kill_events.index:
        current_time = tl_df.loc[idx, 'time']
        
        # 현재 킬 기준 10초 이내의 킬들 찾기
        time_window = (kill_events['time'] >= current_time) & \
                      (kill_events['time'] <= current_time + pd.Timedelta(seconds=10))
        kills_in_window = time_window.sum()
        
        # 3개 이상이면 해당 킬들을 kill_streak으로 마킹
        if kills_in_window >= 3:
            window_indices = kill_events[time_window].index
            tl_df.loc[window_indices, 'kill_streak'] = 1

print(f"총 kill_streak 이벤트 수: {tl_df['kill_streak'].sum()}")

총 kill_streak 이벤트 수: 4538


In [57]:
# tl_df의 time(timedelta)을 df_all의 time 형식(텍스트)으로 변환하는 함수
def timedelta_to_text(td):
    """timedelta를 '-H:MM:SS' 또는 'H:MM:SS' 형식의 텍스트로 변환"""
    if pd.isna(td):
        return None
    
    total_seconds = int(td.total_seconds())
    is_negative = total_seconds < 0
    total_seconds = abs(total_seconds)
    
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    
    if is_negative:
        return f"-{hours}:{minutes:02d}:{seconds:02d}"
    else:
        return f"{hours}:{minutes:02d}:{seconds:02d}"

# df_all의 time(텍스트)을 초 단위로 변환하는 함수
def time_text_to_seconds(time_str):
    """'-H:MM:SS' 형식의 텍스트를 초 단위로 변환"""
    if pd.isna(time_str) or time_str == '':
        return None
    
    time_str = str(time_str).strip()
    is_negative = time_str.startswith('-')
    if is_negative:
        time_str = time_str[1:]
    
    parts = time_str.split(':')
    if len(parts) == 3:
        hours, minutes, seconds = int(parts[0]), int(parts[1]), int(parts[2])
    elif len(parts) == 2:
        hours = 0
        minutes, seconds = int(parts[0]), int(parts[1])
    else:
        return None
    
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return -total_seconds if is_negative else total_seconds

# tl_df의 time을 초 단위로 변환
tl_df['time_seconds'] = tl_df['time'].apply(lambda x: x.total_seconds() if pd.notna(x) else None)

# df_all의 time을 초 단위로 변환
df_all['time_seconds'] = df_all['time'].apply(time_text_to_seconds)

print(f"tl_df time_seconds 샘플: {tl_df['time_seconds'].head().tolist()}")
print(f"df_all time_seconds 샘플: {df_all['time_seconds'].head().tolist()}")

tl_df time_seconds 샘플: [12173.0, 12182.0, 12189.0, 12310.0, 12344.0]
df_all time_seconds 샘플: [-6871, -6143, -5572, -5143, -5125]


In [58]:
# 이벤트 타입 정의 (one-hot 인코딩할 열들)
event_types = ['drake_normal', 'voidgrub', 'herald', 'nashor', 'drake_elder', 
               'kill', 'takedown', 'nexus', 'upset', 'upset_gameend', 'solokill', 'kill_streak']

# 이벤트 열에 '_event' 접미사 추가 (댓글 데이터와 구분)
for event in event_types:
    df_all[f'{event}_event'] = 0

# 날짜별로 이벤트 매핑
from tqdm import tqdm

# tl_df에서 이벤트가 있는 행만 필터링 (최소 하나의 이벤트가 1인 행)
tl_events = tl_df[tl_df[event_types].sum(axis=1) > 0].copy()

print(f"총 이벤트 수: {len(tl_events)}")
print(f"총 댓글 수: {len(df_all)}")

# 날짜별로 처리
for date in tqdm(tl_df['date'].unique(), desc="Processing dates"):
    # 해당 날짜의 이벤트와 댓글 필터링
    date_events = tl_events[tl_events['date'] == date]
    date_comments_mask = df_all['date'] == date
    
    if len(date_events) == 0:
        continue
    
    # 각 이벤트에 대해 시간 범위 내 댓글 매핑
    for _, event_row in date_events.iterrows():
        event_time = event_row['time_seconds']
        if pd.isna(event_time):
            continue
        
        # 이벤트 3초 전 ~ 10초 후 범위
        time_start = event_time - 3
        time_end = event_time + 10
        
        # 해당 시간 범위 내 댓글 찾기
        time_mask = (df_all['time_seconds'] >= time_start) & \
                    (df_all['time_seconds'] <= time_end) & \
                    date_comments_mask
        
        # 각 이벤트 타입에 대해 one-hot 인코딩
        for event in event_types:
            if event_row[event] == 1:
                df_all.loc[time_mask, f'{event}_event'] = 1

# 결과 확인
print("\n=== 이벤트별 매핑된 댓글 수 ===")
for event in event_types:
    count = df_all[f'{event}_event'].sum()
    print(f"{event}_event: {count:,}")

총 이벤트 수: 18880
총 댓글 수: 1520294


Processing dates: 100%|██████████| 96/96 [01:26<00:00,  1.10it/s]


=== 이벤트별 매핑된 댓글 수 ===
drake_normal_event: 30,653
voidgrub_event: 14,791
herald_event: 6,880
nashor_event: 12,131
drake_elder_event: 849
kill_event: 87,582
takedown_event: 26,679
nexus_event: 17,378
upset_event: 14,628
upset_gameend_event: 702
solokill_event: 6,946
kill_streak_event: 21,429


In [59]:
# 결과 샘플 확인
event_cols = [f'{e}_event' for e in event_types]
sample = df_all[df_all[event_cols].sum(axis=1) > 0].sample(min(10, len(df_all[df_all[event_cols].sum(axis=1) > 0])))
print("=== 이벤트가 매핑된 댓글 샘플 ===")
sample[['date', 'time', 'message'] + event_cols]

=== 이벤트가 매핑된 댓글 샘플 ===


,date,time,message,drake_normal_event,voidgrub_event,herald_event,nashor_event,drake_elder_event,kill_event,takedown_event,nexus_event,upset_event,upset_gameend_event,solokill_event,kill_streak_event
932892,2025-08-21,3:05:30,@Yuna Lee -> No disassembly required. [stop sp...,0,1,0,0,0,0,0,0,0,0,0,0
1276891,2025-09-21,1:02:05,신 러,0,1,0,0,0,0,0,0,0,0,0,0
852518,2025-08-28,3:05:17,다된밥에 피터뿌리기!,1,0,0,0,0,0,0,0,0,0,0,0
1007379,2025-08-13,3:24:56,든럼든렇지,0,0,0,0,0,0,0,1,0,0,0,0
548533,2025-04-12,1:18:25,ㅎㅎ,0,0,0,0,0,1,0,0,0,0,0,1
281095,2025-05-09,2:59:04,까는 애들 많다고 역체 논하기에는 좀,1,0,0,0,0,0,0,0,0,0,0,0
71751,2025-05-28,5:21:39,전판안봣나 하,0,0,0,0,0,0,0,1,0,0,0,0
211479,2025-05-15,4:01:22,브리온이랑 자강두천 하는거 실화냐????,0,0,0,0,0,1,1,0,0,0,0,0
1453124,2025-09-10,1:16:07,아 페이커뭐해ㅡㅡ,0,0,0,0,0,1,0,0,0,0,0,0
997402,2025-08-14,4:34:49,먼딜이야ㅋㅋㅋㅋ,0,0,0,0,0,0,0,1,0,0,0,0


In [60]:
# 임시 컬럼 제거 및 결과 저장
df_all = df_all.drop(columns=['time_seconds'], errors='ignore')

# 기존 이벤트 열 제거 (있는 경우) - '_event' 없는 버전
for event in event_types:
    if event in df_all.columns:
        df_all = df_all.drop(columns=[event], errors='ignore')

print(f"최종 df_all 컬럼: {df_all.columns.tolist()}")
print(f"최종 df_all shape: {df_all.shape}")

최종 df_all 컬럼: ['time', 'timestamp', 'author', 'message', 'format', 'date', 'datetime_kst', 'comment_sec', 'date_key', 'drake_normal_event', 'voidgrub_event', 'herald_event', 'nashor_event', 'drake_elder_event', 'kill_event', 'takedown_event', 'nexus_event', 'kill_streak_event', 'solokill_event', 'upset_event', 'upset_gameend_event']
최종 df_all shape: (1520294, 21)


In [62]:
df_all.to_csv("comments_with_events.csv", index=False)